#<font color='#3dc1d3'>  CLINICAL TRIAL - Text extraction based on keyword Search

#Outline
---
# <font color='#3dc1d3'>  
1. Install required library, import and read Dataset
3. Pipeline:
    - Search_keywords_list
    - Extract pdf document using url
    - Search the keyword in the pdf and look for the pdf page with maximum search word count
    - Extract the text for maximum search count
    - Call main funtion apply above functions on dataframe
    
5. Save the information extracted dataset<br>


#1. Install Necesary packages

In [ ]:
!pip install PyPDF2
!pip install simpletransformers
!pip install PdfFileReader
!pip install re
!pip install transformers
!pip install tqdm
!pip install time

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement PdfFileReader (from versions: none)
ERROR: No matching distribution found for PdfFileReader
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement re (from versions: none)
ERROR: No matching distribution found for re
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not fi

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


###Import Libraries

In [ ]:
import json
import PyPDF2
import pandas as pd
import urllib3
import io
from collections import defaultdict
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import string
from nltk.tokenize import RegexpTokenizer
from tqdm import tqdm
import time

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##Read Dataset

In [ ]:
df= pd.read_csv('/content/drive/MyDrive/Clinical Trial/balanced - balanced.csv')
df.head(2)

,Unnamed: 0,Unnamed: 0.1,NCT Number,Status,Conditions,Sponsor,Age,Funded_Bys,Study_Designs,Locations,links
0,0,0,NCT03763474,Completed,diabetes mellitus type 1,aristotle university of thessaloniki,years to years child adult,other,allocation randomized intervention model paral...,endocrine unit of rd department of pediatrics ...,https://ClinicalTrials.gov/ProvidedDocs/74/NCT...
1,1,1,NCT05013294,Completed,diabetes,ku leuven jomo kenyatta university of agricult...,child adult older adult,other,allocation randomized intervention model paral...,jomo kenyatta university of agriculture and te...,https://ClinicalTrials.gov/ProvidedDocs/94/NCT...


In [ ]:
df.shape

(370, 11)

In [ ]:
df['Status'].value_counts()

Completed        193
Not Completed    177
Name: Status, dtype: int64

#2. Information Extraction

##2.1 Search_keywords_list

In [ ]:
search_list = {'objectives', 'Objectives','Purpose', 'purpose','Statistics', 'treatment', 'Treatment','assessment','Assessment', 'Adverse events', 'adverse','ethics','Ethics','summary','Summary' ,'risk','Risk', 'procedure','Procedure','statistics',  'Regulatory', 'Regulatory','ethical','Protocol'}

##2.2 Extract pdf document using url

In [ ]:
def extract_pdf_by_url(url):

    http = urllib3.PoolManager()
    temp = io.BytesIO()
    temp.write(http.request("GET", url).data)
    #result=[]
    #with pdfplumber.open(temp) as pdf:
    doc = PyPDF2.PdfFileReader(temp)
    extracted_text =  search_keyword(doc)
    return extracted_text


##2.3 Search the keyword in the pdf and look for the pdf page with maximum search word count

In [ ]:
from collections import defaultdict

def search_keyword(doc):
          new_results=[]
          
          page_num = []
          occu = []
          new_=[]
          pages = doc.getNumPages()
          #print(pages)
        
          my_dict = {}
          for search in search_list:
            
            list_pages = []
            #print(search)

            for i in range(pages):

              current_page = doc.getPage(i)
              text = current_page.extractText()

              if (re.findall(search,text)):

                count_page = len(re.findall(search.lower(),text.lower()))
              

                list_pages.append((count_page,i))
                page_num.append(i)
                occu.append(count_page)

            if len(list_pages)==0:
                x="not found"
                #d="o"

            else:
                x="word found"
                max_value = max(list_pages)
                #print('Maximum value:', max_value)
                page_needs_to_extract = max_value[1]
                #print(page_needs_to_extract)
                #print(list_pages)
                all_text_for_search = extract_text_from_pdf(doc,page_needs_to_extract)

                my_dict.update({search : all_text_for_search , "page_found" : max_value})

            new_.append(my_dict.copy())



          filter(None, new_)
          new_=[i for i in new_ if i]


          dd = defaultdict(list)

          for d in (new_): # you can list as many input dicts as you want here
            for key, value in d.items():
                dd[key].append(value)

          dfk=dict(dd)
          new_d = {a:list(set(b)) for a, b in dfk.items()}
          #print(new_d)

          new_match={}
          new_match={
                                          
                                          'links': row,
                                          'pages': pages,
                                          'all_text_for_search':new_d
                        }
          new_results.append(new_match.copy()) 
          #print("new result", new_results)
          return new_results

##2.4 Extract the text for maximum search count

In [ ]:
def extract_text_from_pdf(doc,pdf_page_to_extract):
  page1 = doc.getPage(pdf_page_to_extract)
  textpage= page1.extractText()
  return textpage

##2.5 Call main funtion apply above functions on dataframe

In [ ]:
final=[]
i=0
for row in df['links']:

        print(row)
        i=i+1

        print("record number : ",i)
        try:
        
          pages1= extract_pdf_by_url(row)
        except (KeyError,UnicodeEncodeError,NameError,TypeError) as err:
        
          continue
     
        #print(pages1)
        final.append(pages1)



https://ClinicalTrials.gov/ProvidedDocs/74/NCT03763474/Prot_SAP_001.pdf
record number :  1
https://ClinicalTrials.gov/ProvidedDocs/94/NCT05013294/Prot_000.pdf
record number :  2
https://ClinicalTrials.gov/ProvidedDocs/21/NCT03108521/Prot_SAP_000.pdf
record number :  3
https://ClinicalTrials.gov/ProvidedDocs/52/NCT02928952/Prot_SAP_000.pdf
record number :  4
https://ClinicalTrials.gov/ProvidedDocs/31/NCT05106231/Prot_SAP_ICF_001.pdf
record number :  5
https://ClinicalTrials.gov/ProvidedDocs/30/NCT02002130/Prot_SAP_000.pdf
record number :  6
https://ClinicalTrials.gov/ProvidedDocs/22/NCT02641522/Prot_SAP_000.pdf
record number :  7


/usr/local/lib/python3.7/dist-packages/PyPDF2/_reader.py:1345: PdfReadWarning: Xref table not zero-indexed. ID numbers for objects will be corrected.
  PdfReadWarning,


https://ClinicalTrials.gov/ProvidedDocs/34/NCT03224234/Prot_SAP_000.pdf
record number :  8
https://ClinicalTrials.gov/ProvidedDocs/09/NCT03626909/Prot_SAP_000.pdf
record number :  9
https://ClinicalTrials.gov/ProvidedDocs/62/NCT02953262/Prot_SAP_000.pdf
record number :  10
https://ClinicalTrials.gov/ProvidedDocs/06/NCT05187806/Prot_SAP_000.pdf
record number :  11
https://ClinicalTrials.gov/ProvidedDocs/56/NCT03176056/Prot_000.pdf
record number :  12
https://ClinicalTrials.gov/ProvidedDocs/99/NCT03737799/Prot_000.pdf
record number :  13
https://ClinicalTrials.gov/ProvidedDocs/30/NCT01908530/Prot_000.pdf
record number :  14
https://ClinicalTrials.gov/ProvidedDocs/07/NCT01613807/Prot_000.pdf
record number :  15
https://ClinicalTrials.gov/ProvidedDocs/17/NCT04529317/Prot_SAP_ICF_000.pdf
record number :  16
https://ClinicalTrials.gov/ProvidedDocs/10/NCT02889510/Prot_SAP_000.pdf
record number :  17
https://ClinicalTrials.gov/ProvidedDocs/98/NCT05099198/Prot_SAP_000.pdf
record number :  18
ht

/usr/local/lib/python3.7/dist-packages/PyPDF2/_page.py:1406: PdfReadWarning:  impossible to decode XFormObject /Meta13
  PdfReadWarning,


https://ClinicalTrials.gov/ProvidedDocs/13/NCT00756613/Prot_SAP_ICF_000.pdf
record number :  156
https://ClinicalTrials.gov/ProvidedDocs/66/NCT02004366/Prot_SAP_000.pdf
record number :  157
https://ClinicalTrials.gov/ProvidedDocs/55/NCT01901055/Prot_SAP_000.pdf
record number :  158
https://ClinicalTrials.gov/ProvidedDocs/26/NCT02328326/Prot_SAP_000.pdf
record number :  159
https://ClinicalTrials.gov/ProvidedDocs/75/NCT05174975/Prot_SAP_000.pdf
record number :  160
https://ClinicalTrials.gov/ProvidedDocs/08/NCT02762708/Prot_SAP_000.pdf
record number :  161
https://ClinicalTrials.gov/ProvidedDocs/55/NCT02912455/Prot_SAP_000.pdf
record number :  162
https://ClinicalTrials.gov/ProvidedDocs/39/NCT01951339/Prot_SAP_000.pdf
record number :  163
https://ClinicalTrials.gov/ProvidedDocs/36/NCT02671136/Prot_SAP_000.pdf
record number :  164
https://ClinicalTrials.gov/ProvidedDocs/66/NCT02325466/Prot_SAP_000.pdf
record number :  165
https://ClinicalTrials.gov/ProvidedDocs/75/NCT04098575/Prot_SAP_00

...

https://ClinicalTrials.gov/ProvidedDocs/58/NCT04862858/Prot_SAP_000.pdf
record number :  230
https://ClinicalTrials.gov/ProvidedDocs/93/NCT02771093/Prot_000.pdf
record number :  231
https://ClinicalTrials.gov/ProvidedDocs/49/NCT04440449/Prot_SAP_000.pdf
record number :  232
https://ClinicalTrials.gov/ProvidedDocs/33/NCT04369833/Prot_SAP_000.pdf
record number :  233
https://ClinicalTrials.gov/ProvidedDocs/56/NCT01729156/Prot_000.pdf
record number :  234
https://ClinicalTrials.gov/ProvidedDocs/76/NCT02672176/Prot_000.pdf
record number :  235
https://ClinicalTrials.gov/ProvidedDocs/84/NCT02886884/Prot_SAP_000.pdf
record number :  236
https://ClinicalTrials.gov/ProvidedDocs/37/NCT03324737/Prot_001.pdf
record number :  237
https://ClinicalTrials.gov/ProvidedDocs/50/NCT02548650/Prot_SAP_000.pdf
record number :  238
https://ClinicalTrials.gov/ProvidedDocs/14/NCT01980914/Prot_SAP_000.pdf
record number :  239
https://ClinicalTrials.gov/ProvidedDocs/30/NCT04877730/Prot_SAP_000.pdf
record number 

/usr/local/lib/python3.7/dist-packages/PyPDF2/_page.py:1406: PdfReadWarning:  impossible to decode XFormObject /Meta38
  PdfReadWarning,


https://ClinicalTrials.gov/ProvidedDocs/28/NCT03337828/Prot_SAP_000.pdf
record number :  289
https://ClinicalTrials.gov/ProvidedDocs/24/NCT05367024/Prot_SAP_000.pdf
record number :  290
https://ClinicalTrials.gov/ProvidedDocs/62/NCT05249062/Prot_001.pdf
record number :  291
https://ClinicalTrials.gov/ProvidedDocs/94/NCT04127994/ICF_000.pdf
record number :  292
https://ClinicalTrials.gov/ProvidedDocs/82/NCT03827382/Prot_000.pdf
record number :  293
https://ClinicalTrials.gov/ProvidedDocs/64/NCT02387164/Prot_000.pdf
record number :  294
https://ClinicalTrials.gov/ProvidedDocs/51/NCT04742751/Prot_SAP_000.pdf
record number :  295
https://ClinicalTrials.gov/ProvidedDocs/58/NCT03608358/Prot_000.pdf
record number :  296
https://ClinicalTrials.gov/ProvidedDocs/42/NCT03544242/Prot_SAP_000.pdf
record number :  297
https://ClinicalTrials.gov/ProvidedDocs/49/NCT02989649/Prot_000.pdf
record number :  298
https://ClinicalTrials.gov/ProvidedDocs/23/NCT03626623/Prot_SAP_000.pdf
record number :  299
ht

/usr/local/lib/python3.7/dist-packages/PyPDF2/_page.py:1406: PdfReadWarning:  impossible to decode XFormObject /Meta40
  PdfReadWarning,


https://ClinicalTrials.gov/ProvidedDocs/74/NCT05240274/Prot_SAP_000.pdf
record number :  302
https://ClinicalTrials.gov/ProvidedDocs/42/NCT00679042/Prot_000.pdf
record number :  303
https://ClinicalTrials.gov/ProvidedDocs/42/NCT03490942/Prot_000.pdf
record number :  304
https://ClinicalTrials.gov/ProvidedDocs/23/NCT03959423/Prot_000.pdf
record number :  305
https://ClinicalTrials.gov/ProvidedDocs/73/NCT04004273/Prot_002.pdf
record number :  306
https://ClinicalTrials.gov/ProvidedDocs/98/NCT00967798/Prot_SAP_000.pdf
record number :  307
https://ClinicalTrials.gov/ProvidedDocs/71/NCT03132571/Prot_SAP_000.pdf
record number :  308
https://ClinicalTrials.gov/ProvidedDocs/30/NCT05101330/Prot_000.pdf
record number :  309
https://ClinicalTrials.gov/ProvidedDocs/89/NCT02665689/Prot_SAP_000.pdf
record number :  310
https://ClinicalTrials.gov/ProvidedDocs/29/NCT03936829/Prot_000.pdf
record number :  311
https://ClinicalTrials.gov/ProvidedDocs/38/NCT03459638/Prot_SAP_000.pdf
record number :  312
h

In [ ]:
extracted_text_Dataframe= pd.DataFrame(final).stack().apply(pd.Series)
df1= pd.concat([extracted_text_Dataframe.drop(['all_text_for_search'], axis=1), extracted_text_Dataframe['all_text_for_search'].apply(pd.Series)], axis=1)
df1.head(1)

,,links,pages,Treatment,page_found,Protocol,Statistics,Procedure,adverse,Ethics,assessment,...,Risk,Assessment,purpose,risk,Summary,objectives,ethical,Regulatory,Adverse events,Purpose
0,0,https://ClinicalTrials.gov/ProvidedDocs/74/NCT...,3,[Protocol \nChildren and adolescents with T1D...,"[(1, 2), (1, 1)]",[Protocol \nChildren and adolescents with T1D...,[The change of HbA1c values from baseline to ...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#3. Save the information extracted dataset

In [ ]:
import os
os.chdir('/content/drive/MyDrive/Clinical Trial/')
df1.to_csv('information_extraction.csv')
